# Topics in Dynamical Systems
## Exercise 2

In [1]:
using LinearAlgebra
using GLMakie

### Question 1: deterministic case

Deterministic bifurcation diagram for the SODE
$$x' = qx - x^3 + \sigma W'$$

In [11]:
qneg = range(-1, 0, length=100)
qpos = range(0, 1, length=100)
fig_bf = Figure()
ax_bf = Axis(fig_bf[1, 1], xlabel=L"q", ylabel=L"x*", xlabelsize=26, ylabelsize=26, limits = (-1, 1, -1, 1))
lines!(ax_bf, qneg, zeros(100), color = :blue, linestyle=:solid)
lines!(ax_bf, qpos, zeros(100), color = :blue, linestyle=:dash)
lines!(ax_bf, qpos, sqrt.(qpos), color = :orange, linestyle=:solid)
lines!(ax_bf, qpos, -sqrt.(qpos), color = :orange, linestyle=:solid)
fig_bf

### Question 2: Fokker-Plank equation

Fokker-Plank equation:
$$\partial_t \rho = \frac{1}{2}\sigma^2 \partial^2_x \rho - \partial_x ((qx - x^3) \rho)$$

First, discretize the space with forward finite differences and zero Dirichlet boundary conditions:

$$\partial_t \rho_k(t) = \frac{1}{2} \sigma^2 \frac{\rho_{k-1}(t) - 2 \rho_{k}(t) + \rho_{k+1}(t)}{\Delta x^2} - \frac{(qx_{k+1} - x_{k+1}^3) \rho_{k+1}(t) - (q x_k - x_k^3) \rho_{k}(t)}{\Delta x} $$

We obtain a multi-dimensional ODE. The $x^3$-term makes the ODE super stiff, and if we try to solve it with an explicit method, it will explode even at tiny time steps. The remedy is to use an implicit method, for example, implicit Euler:

$$\frac{\rho_{t+1, k} - \rho_{t, k}}{\Delta t} = \frac{1}{2} \sigma^2 \frac{\rho_{t+1, k-1} - 2 \rho_{t+1, k} + \rho_{t+1, k+1}}{\Delta x^2} - \frac{(qx_{k+1} - x_{k+1}^3) \rho_{t+1, k+1}- (q x_k - x_k^3) \rho_{t+1, k}}{\Delta x} $$

Bring all $t+1$ terms to the left and all $t$ terms to the right:
$$%\rho_{t+1, k} = \rho_{t, k} + \Delta t \left( \frac{1}{2} \sigma^2 \frac{\rho_{t, k-1} - 2 \rho_{t, k} + \rho_{t, k+1}}{\Delta x^2} - \frac{(qx_{k+1} - x_{k+1}^3) \rho_{t, k+1} - (q x_k - x_k^3) \rho_{t, k}}{\Delta x} \right)$$

$$\rho_{t+1, k} - \frac{1}{2} \sigma^2 \frac{\Delta t}{\Delta x^2} \left( \rho_{t+1, k-1} - 2 \rho_{t+1, k} + \rho_{t+1, k+1}\right) + \frac{\Delta t}{\Delta x} \left((qx_{k+1} - x_{k+1}^3) \rho_{t+1, k+1}- (q x_k - x_k^3) \rho_{t+1, k}\right) = \rho_{t, k}$$

$$A \bm{\rho}_{t+1} = \bm{\rho}_t$$

we get a linear equation for $\bm{\rho}_{t+1}$ where matrix $A$ is tridiagonal with the stencil

$$
\begin{bmatrix}
- \frac{1}{2}\sigma^2 \frac{\Delta t}{\Delta x^2} &
1 + \sigma^2 \frac{\Delta t}{\Delta x^2} - \frac{\Delta t}{\Delta x} (q x_k - x_k^3) &
- \frac{1}{2}\sigma^2 \frac{\Delta t}{\Delta x^2} + \frac{\Delta t}{\Delta x} (q x_{k+1} - x_{k+1}^3)
\end{bmatrix}
$$


#### Simulation

Define the parameters:

In [21]:
L = 10   # x-interval x ∈ [-L, L]
Δx = 0.001
Δt = 0.001
xs = -L:Δx:L
xinner = xs[2:end-1]
sigma = 1
N = length(xs)  # number of points in x
num_steps = 50000    # max number of time steps
buffer = 100;     # number of steps between updating the visualization
rho = Observable(zeros(N));

Create the plot:

In [22]:
fig = Figure()
ax = Axis(fig[1, 1], 
          xlabel = L"x",
          xlabelsize=20,
          ylabel = L"probability density $\rho(t, x)$",
          ylabelsize=20,
          xautolimitmargin=(0.0, 0.0))

# plot the solution of the pde
lines!(ax, xs, rho)
autolimits!(ax)

# add some interactivity
# slider for q
slider_params = (range = -1.:0.01:1., label="q", startvalue=-1.)
sg = SliderGrid(fig[2, 1], slider_params)
q = lift(q -> q, sg.sliders[1].value)

# button to stop the simulation
isrunning = Observable(false)
button = Button(fig[3, 1], label="Play", tellwidth=false)
on(button.clicks) do clicks
    isrunning[] = !isrunning[]
    button.label = isrunning[] ? "Pause" : "Play"
end
on(button.clicks) do clicks
    @async while isrunning[]
        # stop the computations if the figure is closed
        if !isopen(fig.scene)
            break
        end
        step!(rho, A, isrunning)
        autolimits!(ax)
    end
end

# plot the stable points as dashed lines
stable_points = @lift $q > 0 ? [sqrt($q), -sqrt($q)] : [0., ]
vlines!(ax, stable_points, color=:red, linestyle=:dash)

display(fig)

GLMakie.Screen(...)

Stamp the matrix: 

In [23]:
A = lift(q) do q
    lower_diag = - 0.5 * sigma^2 * Δt / Δx^2 * ones(N - 3)
    main_diag = @. 1 + sigma^2 * Δt / Δx^2 - Δt / Δx * (q[] * xinner - xinner^3)
    upper_diag = @. - 0.5 * sigma^2 * Δt / Δx^2 + Δt / Δx * (q[] * xinner[2:end] - xinner[2:end]^3)
    Tridiagonal(lower_diag, main_diag, upper_diag)
end;

Set the initial condition to some distribution:

In [24]:
# uniform distribution in the interval -0.5:0.5
box(x) = abs(x) > 0.5 ? 0. : 1.

# uniform distribution in the interval -L:L
uniform(x) = 1/(2*L) 

# normal distribution with mean μ and std σ
normal(x; μ=0, σ=1) = 1/sqrt(2π)/σ * exp(-0.5 * (x - μ)^2/σ^2);

# set up the initial condition ρ(0, x) = ρ0(x)
#rho[] = normal.(xs; μ=1.5, σ=0.1)
#rho[] = box.(xs)
rho[] = uniform.(xs)
rho[][1] = 0.
rho[][end] = 0.

0.0

Run the simulation

In [25]:
function step!(rho, A, isrunning)
    rho_buffer = rho[][2:end-1]
    for i in 1:buffer
        rho_buffer = A[] \ rho_buffer
    end
    rho[][2:end-1] = rho_buffer
    notify(rho)
    # abort if the system becomes unstable
    if maximum(rho_buffer) > 100
        println("Max value has exceeded the threshold. Panic! 😱")
        isrunning[] = false
    end
    sleep(0.001)
end

step! (generic function with 1 method)